In [1]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report, f1_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np 


In [2]:
dataset = pd.read_csv("Malware_Dataset.csv")

In [3]:
dataset.head()

,Category,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.nprocs64bit,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,...,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.interactive_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric,Class
0,Benign,45,17,10.555556,0,202.844444,1694,38.500000,9129,212.302326,...,221,26,24,116,0,121,87,0,8,Benign
1,Benign,47,19,11.531915,0,242.234043,2074,44.127660,11385,242.234043,...,222,26,24,118,0,122,87,0,8,Benign
2,Benign,40,14,14.725000,0,288.225000,1932,48.300000,11529,288.225000,...,222,26,27,118,0,120,88,0,8,Benign
3,Benign,32,13,13.500000,0,264.281250,1445,45.156250,8457,264.281250,...,222,26,27,118,0,120,88,0,8,Benign
4,Benign,42,16,11.452381,0,281.333333,2067,49.214286,11816,281.333333,...,222,26,24,118,0,124,87,0,8,Benign


In [4]:
df = dataset.drop(['Category'], axis=1)

In [5]:
df['Class'] = df['Class'].replace(['Malware', 'Benign'], [1, 0]) 

In [7]:
malware_samples_indices = df[df['Class'] == 1].index

In [18]:
num_rows_to_delete = int(0.8 * len(malware_samples_indices))

In [19]:
print(num_rows_to_delete)

23438


In [20]:
rows_to_delete = np.random.choice(malware_samples_indices, size=num_rows_to_delete, replace=False)

In [21]:
df = df.drop(rows_to_delete)

KeyError: '[43433, 46982, 42544, 54203, 47115, 39746, 44370, 55281, 33676, 29370, 34468, 49440, 41354, 45151, 48240, 33816, 50877, 38166, 52509, 57557, 30821, 30810, 37838, 30868, 56305, 32084, 40971, 33509, 52813, 38395, 45881, 44124, 47326, 31389, 31238, 55071, 49323, 57866, 38145, 50858, 40290, 50370, 56871, 38588, 42976, 57333, 47263, 46969, 32010, 40542, 40381, 33222, 36693, 48103, 45536, 36587, 40941, 43254, 45001, 48649, 44254, 44618, 41398, 42374, 50568, 43183, 38228, 38473, 56020, 46937, 55275, 45629, 34105, 31386, 52258, 47038, 50511, 31064, 43551, 36691, 57017, 32395, 34355, 43203, 57917, 33317, 46459, 42362, 54116, 48376, 54547, 45847, 34745, 47979, 50555, 49353, 54569, 34188, 40506, 36869, 52274, 38624, 39894, 57834, 56049, 44500, 47335, 46589, 55641, 32087, 42652, 37284, 34875, 32628, 45734, 50271, 38193, 40850, 49309, 56266, 36154, 40274, 40446, 38455, 49254, 50484, 40431, 29474, 50424, 39372, 31254, 40612, 53496, 49828, 53384, 54611, 35500, 39611, 42784, 41525, 33019, 5

In [14]:
print(df['Class'].value_counts())

0    29298
1    20509
Name: Class, dtype: int64


In [16]:
df.shape

(49807, 56)

NameError: name 'X' is not defined